## a JSON format compatible with the Pandas data structure
### introduction
The data type is not explicitely taken into account in the current JSON interface.     
    
The existing solution is to use a data schema in addition.

### proposal
To have a simple and compact solution, I propose to use the [JSON-NTV format (Named and Typed Value)](https://github.com/loco-philippe/NTV#readme) - which integrates the notion of type - and its JSON-TAB variation for tabular data.

### content
This NoteBook uses examples to present some key points

In [1]:
from json_ntv import NtvSingle, NtvList, Ntv, NtvConnector, NtvType, Namespace
from datetime import date, datetime
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(compact=True, width=100)
pr = pp.pprint
option = {'encode_format': 'obj', 'dicobj': {'field': 'SeriesConnec', 'tab': 'DataFrameConnec'}}

In [2]:
s_obj = pd.Series(["a", "b", "c", "a"], dtype="category")
i_obj = pd.Series([1,2,1,2], dtype="category")

print('categorical object Series  : ', Ntv.obj(s_obj))
print('categorical integer Series : ', Ntv.obj(i_obj))

categorical object Series  :  {":field": [["a", "b", "c"], [0, 1, 2, 0]]}
categorical integer Series :  {":field": [[1, 2], [0, 1, 0, 1]]}


In [3]:
print(s_obj.equals(Ntv.obj(s_obj).to_obj(**option)))
print(i_obj.equals(Ntv.obj(i_obj).to_obj(**option)))

True
True


In [4]:
s_obj = pd.Series(["a", "b", "c", "a"], name='string', dtype="category")
i_obj = pd.Series([1,2,1,2], name='integer', dtype="category")

print('categorical object Series  : ', Ntv.obj(s_obj))
print('categorical integer Series : ', Ntv.obj(i_obj))

categorical object Series  :  {":field": {"string": [["a", "b", "c"], [0, 1, 2, 0]]}}
categorical integer Series :  {":field": {"integer": [[1, 2], [0, 1, 0, 1]]}}


In [5]:
print(s_obj.equals(Ntv.obj(s_obj).to_obj(**option)))
print(i_obj.equals(Ntv.obj(i_obj).to_obj(**option)))

True
True


In [6]:
s_df = pd.DataFrame({"A": list("abca"), "B": list("bccd")}, dtype="category")

print('categorical object DataFrame  : ')
print(Ntv.obj(s_df))

categorical object DataFrame  : 
{":tab": {"index": [0, 1, 2, 3], "A": [["a", "b", "c"], [0, 1, 2, 0]], "B": [["b", "c", "d"], [0, 1, 1, 2]]}}


In [7]:
print(s_df.equals(Ntv.obj(s_df).to_obj(**option)))

True


In [21]:


field_data = {'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21']}
tab_data   = {'index':           [1, 2, 3],
              'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21'], 
              'value':           [10, 20, 30],
              'value32::int32':  [10, 20, 30],
              'coord::point':    [[1,2], [3,4], [5,6]],
              'names::string':   ['john', 'eric', 'judith']}

field = Ntv.obj({':field': field_data})
tab   = Ntv.obj({':tab'  : tab_data})

# the DataFrame Connector is associated with NtvType 'tab' in dicobj 
sr = field.to_obj(encode_format='obj', dicobj={'field': 'SeriesConnec'})
df = tab.to_obj  (encode_format='obj', dicobj={'tab': 'DataFrameConnec'})

# pandas dtype conform to Ntv type
print(sr, '\n')
print(df.dtypes)
df

0   1964-01-01
1   1985-02-05
2   2022-01-21
Name: dates, dtype: datetime64[ns] 

dates      datetime64[ns]
value               int64
value32             int32
coord              object
names              string
dtype: object


C:\Opensource\anaconda3\envs\observ_v10_20230430\lib\site-packages\pandas\core\dtypes\cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,dates,value,value32,coord,names
index,,,,,
1,1964-01-01,10,10,POINT (1 2),john
2,1985-02-05,20,20,POINT (3 4),eric
3,2022-01-21,30,30,POINT (5 6),judith


In [23]:
# the dataframe generated from JSON-NTV data is identical to the initial dataframe
df2 = Ntv.obj(df).to_obj(**option)
print(df2.dtypes)
df2

dates      datetime64[ns]
value               int64
value32             int32
coord              object
names              string
dtype: object


,dates,value,value32,coord,names
index,,,,,
1,1964-01-01,10,10,POINT (1 2),john
2,1985-02-05,20,20,POINT (3 4),eric
3,2022-01-21,30,30,POINT (5 6),judith


In [10]:

data   = {'index':           [100, 200, 300, 400, 500, 600],
          'dates::date':     ['1964-01-01', '1985-02-05', '2022-01-21', '1964-01-01', '1985-02-05', '2022-01-21'], 
          'value':           [10, 10, 20, 20, {'valid?': 30}, 30],
          'value32::int32':  [12, 12, 22, 22, 32, 32],
          'res':             {'res1': 10, 'res2': 20, 'res3': 30, 'res4': 10, 'res5': 20, 'res6': 30},
          'coord::point':    [[1,2], [3,4], [5,6], [7,8], {'same as 2nd point': [3,4]}, [5,6]],
          'names::string':   ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
          'unique:boolean':  True }

df  = Ntv.obj({':tab'  : data}).to_obj  (encode_format='obj', dicobj={'tab': 'DataFrameConnec'})

# preservation of type
print(df.dtypes)

# preservation of index
df

dates      datetime64[ns]
value               int64
value32             int32
res                 int64
coord              object
names              string
unique            boolean
dtype: object


C:\Opensource\anaconda3\envs\observ_v10_20230430\lib\site-packages\pandas\core\dtypes\cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,dates,value,value32,res,coord,names,unique
index,,,,,,,
100,1964-01-01,10,12,10,POINT (1 2),john,True
200,1985-02-05,10,12,20,POINT (3 4),eric,True
300,2022-01-21,20,22,30,POINT (5 6),judith,True
400,1964-01-01,20,22,10,POINT (7 8),mila,True
500,1985-02-05,30,32,20,POINT (3 4),hector,True
600,2022-01-21,30,32,30,POINT (5 6),maria,True


In [11]:
from observation import Ilist

ntv2 = Ilist.ntv(Ntv.obj(df).val).to_ntv('full')
print(ntv2)
df2 = Ntv.obj({':tab': ntv2.to_obj()}).to_obj(encode_format='obj', dicobj={'tab': 'DataFrameConnec'})

print(df2.dtypes)
df2

{"index": [100, 200, 300, 400, 500, 600], "dates::datetime": ["1964-01-01T00:00:00.000", "1985-02-05T00:00:00.000", "2022-01-21T00:00:00.000", "1964-01-01T00:00:00.000", "1985-02-05T00:00:00.000", "2022-01-21T00:00:00.000"], "value": [10, 10, 20, 20, 30, 30], "value32::int32": [12, 12, 22, 22, 32, 32], "res": [10, 20, 30, 10, 20, 30], "coord": ["POINT (1 2)", "POINT (3 4)", "POINT (5 6)", "POINT (7 8)", "POINT (3 4)", "POINT (5 6)"], "names::string": ["john", "eric", "judith", "mila", "hector", "maria"], "unique": true}
dates      datetime64[ns]
value               int64
value32             int32
res                 int64
coord              object
names              string
unique               bool
dtype: object


,dates,value,value32,res,coord,names,unique
index,,,,,,,
100,1964-01-01,10,12,10,POINT (1 2),john,True
200,1985-02-05,10,12,20,POINT (3 4),eric,True
300,2022-01-21,20,22,30,POINT (5 6),judith,True
400,1964-01-01,20,22,10,POINT (7 8),mila,True
500,1985-02-05,30,32,20,POINT (3 4),hector,True
600,2022-01-21,30,32,30,POINT (5 6),maria,True


In [12]:
from observation import Ilist

ntv2 = Ilist.ntv(Ntv.obj(df).val).to_ntv('default')
print(ntv2)
#print(Ilist.ntv(ntv2).to_ntv('full'))
df3 = Ntv.obj({':tab': Ilist.ntv(ntv2).to_ntv('full').to_obj()}).to_obj(encode_format='obj', dicobj={'tab': 'DataFrameConnec'})

print(df3.dtypes)
df3

{"index": [100, 200, 300, 400, 500, 600], "dates": [{"::datetime": ["2022-01-21T00:00:00.000", "1964-01-01T00:00:00.000", "1985-02-05T00:00:00.000"]}, [1, 2, 0, 1, 2, 0]], "value": [[30, 20, 10], [2, 2, 1, 1, 0, 0]], "value32": [{"::int32": [12, 22, 32]}, [0, 0, 1, 1, 2, 2]], "res": [[30, 20, 10], [2, 1, 0, 2, 1, 0]], "coord": [["POINT (5 6)", "POINT (1 2)", "POINT (3 4)", "POINT (7 8)"], [1, 2, 0, 3, 2, 0]], "names::string": ["john", "eric", "judith", "mila", "hector", "maria"], "unique": true}
dates      datetime64[ns]
value               int64
value32             int32
res                 int64
coord              object
names              string
unique               bool
dtype: object


,dates,value,value32,res,coord,names,unique
index,,,,,,,
100,1964-01-01,10,12,10,POINT (1 2),john,True
200,1985-02-05,10,12,20,POINT (3 4),eric,True
300,2022-01-21,20,22,30,POINT (5 6),judith,True
400,1964-01-01,20,22,10,POINT (7 8),mila,True
500,1985-02-05,30,32,20,POINT (3 4),hector,True
600,2022-01-21,30,32,30,POINT (5 6),maria,True


In [13]:
from observation import Ilist

ntv2 = Ilist.ntv(Ntv.obj(df).val).to_ntv()
print(ntv2)
#print(Ilist.ntv(ntv2).to_ntv('full'))
df3 = Ntv.obj({':tab': Ilist.ntv(ntv2).to_ntv('full').to_obj()}).to_obj(encode_format='obj', dicobj={'tab': 'DataFrameConnec'})

print(df3.dtypes)
df3

{"index": [100, 200, 300, 400, 500, 600], "dates": [{"::datetime": ["2022-01-21T00:00:00.000", "1964-01-01T00:00:00.000", "1985-02-05T00:00:00.000"]}, 5, [0, 1, 2, 1]], "value": [[30, 20, 10], [2, 2, 1, 1, 0, 0]], "value32": [{"::int32": [32, 22, 12]}, 2], "res": [[30, 10, 20], 1], "coord": [["POINT (5 6)", "POINT (1 2)", "POINT (3 4)", "POINT (7 8)"], [1, 2, 0, 3, 2, 0]], "names::string": ["john", "eric", "judith", "mila", "hector", "maria"], "unique": true}
dates      datetime64[ns]
value               int64
value32             int32
res                 int64
coord              object
names              string
unique               bool
dtype: object


,dates,value,value32,res,coord,names,unique
index,,,,,,,
100,1964-01-01,10,12,10,POINT (1 2),john,True
200,1985-02-05,10,12,20,POINT (3 4),eric,True
300,2022-01-21,20,22,30,POINT (5 6),judith,True
400,1964-01-01,20,22,10,POINT (7 8),mila,True
500,1985-02-05,30,32,20,POINT (3 4),hector,True
600,2022-01-21,30,32,30,POINT (5 6),maria,True


In [14]:
data   = {'index':           [10, 20, 30],
              'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21'], 
              'value':           [10, 20, {'valid?': 30}],
              'value32::int32':  [10, 20, 30],
              'res':             {'res1': 10, 'res2': 20, 'res3': 30},
              'coord::point':    [[1,2], [3,4], [5,6]],
              'names::string':   ['john', 'eric', 'judith'],
              'unique:boolean': True}
df  = Ntv.obj({':tab'  : data}).to_obj  (encode_format='obj', dicobj={'tab': 'DataFrameConnec'})
print(df.dtypes)
df


dates      datetime64[ns]
value               int64
value32             int32
res                 int64
coord              object
names              string
unique            boolean
dtype: object


C:\Opensource\anaconda3\envs\observ_v10_20230430\lib\site-packages\pandas\core\dtypes\cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,dates,value,value32,res,coord,names,unique
index,,,,,,,
10,1964-01-01,10,10,10,POINT (1 2),john,True
20,1985-02-05,20,20,20,POINT (3 4),eric,True
30,2022-01-21,30,30,30,POINT (5 6),judith,True
